In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Edward version:", ed.__version__)
from edward.models import Multinomial, Normal

TensorFlow version: 1.2.1
Edward version: 1.3.3


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('datasets/MNIST_data', one_hot=True)
n_train_samples = mnist.train.num_examples
n_test_samples = mnist.test.num_examples
print("Number of training examples:", n_train_samples)
print("Number of testing examples:", n_test_samples)

Extracting datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
Number of training examples: 55000
Number of testing examples: 10000


### Define the model

In [3]:
def forward(X, W_0, b_0, W_1, b_1):
    hidden_layer = tf.nn.relu(tf.matmul(X, W_0) + b_0)
    output = tf.matmul(hidden_layer, W_1) + b_1
    return output

### Priors

In [4]:
num_features = 784
num_classes = 10
hidden_units = 500

W_0 = Normal(loc=tf.zeros([num_features, hidden_units]), scale=tf.ones([num_features, hidden_units]))
b_0 = Normal(loc=tf.zeros(hidden_units), scale=tf.ones(hidden_units))
W_1 = Normal(loc=tf.zeros([hidden_units, num_classes]), scale=tf.ones([hidden_units, num_classes]))
b_1 = Normal(loc=tf.zeros(num_classes), scale=tf.ones(num_classes))

In [5]:
X, y_train = mnist.train.next_batch(2000)
y = Multinomial(logits=forward(X, W_0, b_0, W_1, b_1), total_count=1.)
#y_train = mnist.train.labels

In [6]:
# Posteriors
qw_0_loc = tf.Variable(tf.random_normal([num_features, hidden_units]))
qw_0_scale = tf.nn.softplus(tf.Variable(tf.random_normal([num_features, hidden_units])))
qb_0_loc = tf.Variable(tf.random_normal([hidden_units]) + 5)
qb_0_scale = tf.nn.softplus(tf.Variable(tf.random_normal([hidden_units])))

qw_0 = Normal(loc=qw_0_loc, scale=qw_0_scale)
qb_0 = Normal(loc=qb_0_loc, scale=qb_0_scale)

qw_1_loc = tf.Variable(tf.random_normal([hidden_units, num_classes]))
qw_1_scale = tf.nn.softplus(tf.Variable(tf.random_normal([hidden_units, num_classes])))
qb_1_loc = tf.Variable(tf.random_normal([num_classes]) + 5)
qb_1_scale = tf.nn.softplus(tf.Variable(tf.random_normal([num_classes])))

qw_1 = Normal(loc=qw_1_loc, scale=qw_1_scale)
qb_1 = Normal(loc=qb_1_loc, scale=qb_1_scale)

In [7]:
inference = ed.KLqp({W_0: qw_0, b_0: qb_0, W_1: qw_1, b_1: qb_1}, data={y: y_train})
inference.run(n_iter=2000, n_samples=50)

   1/2000 [  0%]                                ETA: 182091s | Loss: nan

KeyboardInterrupt: 